## A quick simulator for Axis & Allies Board Game battles

In [1]:
from collections import Counter
import random

In [7]:
# This is the reference for the attack/defense strength of each troop type
# Add more troops here
reference_strength = {'infantry':(1,2), 'tank':(3,3), 'artillery':(2,2)}

In [8]:
def simulate_attack(attacker, defender, verbose = True):
    round = 1
    while (sum(attacker.values()) > 0) and (sum(defender.values()) > 0):
        if verbose:
            print 'Round {}'.format(round)
            print 'Attacker:'
            print attacker
            print 'Defender:'
            print defender
        
        # use a temp variable to cache the values of the defender
        temp_defender = dict(defender)
        
        for troop_type in attacker:
            for i in range(attacker[troop_type]):
                if roll_dice() <= reference_strength[troop_type][0]:
                    defender, type_killed = remove_casualty(defender)
                    if verbose:
                        print troop_type + ' hit ' + type_killed
        for troop_type in temp_defender:
            for i in range(temp_defender[troop_type]):
                if roll_dice() <= reference_strength[troop_type][1]:
                    attacker, type_killed = remove_casualty(attacker)
                    if verbose:
                        print troop_type + ' hit ' + type_killed
        round = round + 1
        
    # Check the results to see if it was a tie
    if sum(attacker.values()) == sum(defender.values()) :
        return 0
    return 1 if sum(attacker.values()) == 0 else -1
    

In [9]:
def roll_dice():
    return random.choice(range(6))

In [10]:
roll_dice()

0

We assume that casualties will be removed in the ascending order of the sum of their attack and defense strength.

In [11]:
def remove_casualty(army):
    if sum(army.values()) <= 0:
        return army, 'nothing'
    weakest_link = ''
    current_weakest_score = 100
    for troop_type in army:
        if sum(reference_strength[troop_type]) < current_weakest_score:
            weakest_link = troop_type
            current_weakest_score = sum(reference_strength[troop_type])
    army[weakest_link] = army[weakest_link] - 1
    if army[weakest_link] == 0:
        army.pop(weakest_link, None)
    return army, weakest_link
        

In [12]:
simulate_attack({'infantry':2, 'tank':3, 'artillery':5}, {'infantry':2, 'tank':3, 'artillery':5})

Round 1
Attacker:
{'infantry': 2, 'tank': 3, 'artillery': 5}
Defender:
{'infantry': 2, 'tank': 3, 'artillery': 5}
tank hit infantry
tank hit infantry
artillery hit artillery
artillery hit artillery
tank hit infantry
artillery hit infantry
artillery hit artillery
Round 2
Attacker:
{'tank': 3, 'artillery': 4}
Defender:
{'tank': 3, 'artillery': 3}
tank hit artillery
tank hit artillery
tank hit artillery
artillery hit tank
artillery hit tank
artillery hit artillery
tank hit artillery
tank hit artillery
tank hit artillery
Round 3
Attacker:
{'tank': 3}
Defender:
{'tank': 1}
tank hit tank
tank hit nothing
tank hit tank


-1

In [13]:
# Simulate n attacks
def simulate_multiple_attacks(attacker, defender, n):
    attack_results = [simulate_attack(dict(attacker), dict(defender), verbose = False) for i in range(n)]
    return Counter(attack_results)
    

In [14]:
simulate_multiple_attacks({'infantry':2, 'tank':3, 'artillery':5}, {'infantry':2, 'tank':3, 'artillery':5}, 10000)

Counter({-1: 4184, 0: 480, 1: 5336})